In [1]:
# machine learning model for NFL 2007 season
import pickle
import pandas as pd

In [2]:
# load the csv file
# df = pd.read_csv('./pbp_csv/play_by_play_2007.csv')
# make dataframe years 1999-2022
df = pd.DataFrame()
for year in range(2011, 2023):
    year_df = pd.read_csv(f'./pbp_csv/play_by_play_{year}.csv', low_memory=False)
    df = pd.concat([df, year_df])

# reindex the dataframe
df = df.reset_index(drop=True)

df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2011_01_ATL_CHI,2011091105,CHI,ATL,REG,1,NaN,NaN,NaN,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36,2011_01_ATL_CHI,2011091105,CHI,ATL,REG,1,CHI,home,ATL,...,0,1,0.008777,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,69,2011_01_ATL_CHI,2011091105,CHI,ATL,REG,1,CHI,home,ATL,...,0,1,0.160478,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,91,2011_01_ATL_CHI,2011091105,CHI,ATL,REG,1,CHI,home,ATL,...,0,1,-1.026969,NaN,NaN,NaN,NaN,NaN,0.441336,-44.133604
4,112,2011_01_ATL_CHI,2011091105,CHI,ATL,REG,1,CHI,home,ATL,...,0,1,0.349349,1.153292,7.155267,6.0,0.620025,0.120212,0.606305,39.369512


In [3]:
# # calculate the team's win percentage
# # format by_date as 'yyyy-mm-dd'
# def win_percentage(df, team, by_date):
#     # get the team's games
#     team_wins = 0
#     team_losses = 0
#     # loop through the games
#     for index, row in df.iterrows():
#         # check if the game is before the date
#         if row['game_date'] >= by_date:
#             break
#         if row['desc'] == 'END GAME':
#             if row['home_team'] == team:
#                 if row['home_score'] > row['away_score']:
#                     team_wins += 1
#                 else:
#                     team_losses += 1
#             elif row['away_team'] == team:
#                 if row['away_score'] > row['home_score']:
#                     team_wins += 1
#                 else:
#                     team_losses += 1
#     # calculate the win percentage
#     if (team_wins + team_losses) == 0:
#         win_percentage = 0.0
#     else:
#         win_percentage = team_wins / (team_wins + team_losses)
#     return win_percentage

# calculate the team's win percentage
def win_percentage(df, team, by_date, year):
    # # get the team's games after september 1st of the year and before the by_date
    # start_date = pd.to_datetime(f'{year}-09-01')
    # games = df[(df['game_date'] >= start_date) & (df['game_date'] < by_date)]
    # home_games = games[games['home_team'] == team]
    # away_games = games[games['away_team'] == team]
    
    # # calculate team's wins and losses
    # home_wins = (home_games['desc'] == 'END GAME') & (home_games['home_score'] > home_games['away_score'])
    # away_wins = (away_games['desc'] == 'END GAME') & (away_games['away_score'] > away_games['home_score'])
    # team_wins = home_wins.sum() + away_wins.sum()

    # home_losses = (home_games['desc'] == 'END GAME') & (home_games['home_score'] <= home_games['away_score'])
    # away_losses = (away_games['desc'] == 'END GAME') & (away_games['away_score'] <= away_games['home_score'])
    # team_losses = home_losses.sum() + away_losses.sum()

    # # calculate the win percentage
    # if (team_wins + team_losses) == 0:
    #     win_percentage = 0.0
    # else:
    #     win_percentage = team_wins / (team_wins + team_losses)
    # return win_percentage
    
    return 0.0


In [6]:
def add_win_percentage(df):
    # convert game_date to datetime if it's not
    if df['game_date'].dtype == 'object':
        df['game_date'] = pd.to_datetime(df['game_date'])

    # create a new column
    df['home_win_percentage'] = 0.0
    df['away_win_percentage'] = 0.0

    team_wls = list(df['home_team'].unique())

    # loop through the games and add the win percentage for each teams prior to the game
    df['home_win_percentage'] = df.apply(lambda row: win_percentage(df, row['home_team'], row['game_date'], row['game_id'][:4]), axis=1)
    df['away_win_percentage'] = df.apply(lambda row: win_percentage(df, row['away_team'], row['game_date'], row['game_id'][:4]), axis=1)
    
    return df

In [7]:
df = add_win_percentage(df)
df.tail()

# df to pkl for times sake
print('dumping to pickle...')
# df.to_pickle('./pbp_2007_WL.pkl')
df.to_pickle('./pbp_1999-2023_WL.pkl')
print('done')

# CHECKPOINT

In [ ]:
import pandas as pd
# load pkl
df = pd.read_pickle('./pbp_2007_WL.pkl')
df.tail()

In [ ]:
columns_to_be_dropped = [
    'game_id',
    'old_game_id', 
    'desc', 
    'play_id',
    'game_date',
    'ydsnet',
    'no_score_prob',
    'opp_fg_prob',
    'opp_safety_prob',
    'opp_td_prob',
    'fg_prob',
    'safety_prob',
    'td_prob',
    'extra_point_prob',
    'two_point_conversion_prob',
    'ep',
    'epa',
    'total_home_epa',
    'total_away_epa',
    'total_home_rush_epa',
    'total_away_rush_epa',
    'total_home_pass_epa',
    'total_away_pass_epa',
    'air_epa',
    'yac_epa',
    'comp_air_epa',
    'comp_yac_epa',
    'total_home_comp_air_epa',
    'total_away_comp_air_epa',
    'total_home_comp_yac_epa',
    'total_away_comp_yac_epa',
    'total_home_raw_air_epa',
    'total_away_raw_air_epa',
    'total_home_raw_yac_epa',
    'total_away_raw_yac_epa',
    'wp',
    'def_wp',
    'home_wp',
    'away_wp',
    'wpa',
    'vegas_wpa',
    'vegas_home_wpa',
    'home_wp_post',
    'away_wp_post',
    'vegas_wp',
    'vegas_home_wp',
    'total_home_rush_wpa',
    'total_away_rush_wpa',
    'total_home_pass_wpa',
    'total_away_pass_wpa',
    'air_wpa',
    'yac_wpa',
    'comp_air_wpa',
    'comp_yac_wpa',
    'total_home_comp_air_wpa',
    'total_away_comp_air_wpa',
    'total_home_comp_yac_wpa',
    'total_away_comp_yac_wpa',
    'total_home_raw_air_wpa',
    'total_away_raw_air_wpa',
    'total_home_raw_yac_wpa',
    'total_away_raw_yac_wpa',
    'passer_player_id',
    'passer_player_name',
    'receiver_player_id',
    'receiver_player_name',
    'rusher_player_id',
    'rusher_player_name',
    'lateral_receiver_player_id',
    'lateral_receiver_player_name',
    'lateral_rusher_player_id',
    'lateral_rusher_player_name',
    'lateral_sack_player_id',
    'lateral_sack_player_name',
    'interception_player_id',
    'interception_player_name',
    'lateral_interception_player_id',
    'lateral_interception_player_name',
    'punt_returner_player_id',
    'punt_returner_player_name',
    'lateral_punt_returner_player_id',
    'lateral_punt_returner_player_name',
    'kickoff_returner_player_name',
    'kickoff_returner_player_id',
    'lateral_kickoff_returner_player_id',
    'lateral_kickoff_returner_player_name',
    'punter_player_id',
    'punter_player_name',
    'kicker_player_name',
    'kicker_player_id',
    'own_kickoff_recovery_player_id',
    'own_kickoff_recovery_player_name',
    'blocked_player_id',
    'blocked_player_name',
    'tackle_for_loss_1_player_id',
    'tackle_for_loss_1_player_name',
    'tackle_for_loss_2_player_id',
    'tackle_for_loss_2_player_name',
    'qb_hit_1_player_id',
    'qb_hit_1_player_name',
    'qb_hit_2_player_id',
    'qb_hit_2_player_name',
    'forced_fumble_player_1_player_id',
    'forced_fumble_player_1_player_name',
    'forced_fumble_player_2_player_id',
    'forced_fumble_player_2_player_name',
    'solo_tackle_1_player_id',
    'solo_tackle_1_player_name',
    'solo_tackle_2_player_id',
    'solo_tackle_2_player_name',
    'assist_tackle_1_player_id',
    'assist_tackle_1_player_name',
    'assist_tackle_2_player_id',
    'assist_tackle_2_player_name',
    'assist_tackle_3_player_id',
    'assist_tackle_3_player_name',
    'assist_tackle_4_player_id',
    'assist_tackle_4_player_name',
    'tackle_with_assist_1_player_id',
    'tackle_with_assist_1_player_name',
    'tackle_with_assist_2_player_id',
    'tackle_with_assist_2_player_name',
    'pass_defense_1_player_id',
    'pass_defense_1_player_name',
    'pass_defense_2_player_id',
    'pass_defense_2_player_name',
    'fumbled_1_player_id',
    'fumbled_1_player_name',
    'fumbled_2_player_id',
    'fumbled_2_player_name',
    'fumble_recovery_1_player_id',
    'fumble_recovery_1_player_name',
    'fumble_recovery_2_player_id',
    'fumble_recovery_2_player_name',
    'sack_player_id',
    'sack_player_name',
    'half_sack_1_player_id',
    'half_sack_1_player_name',
    'half_sack_2_player_id',
    'half_sack_2_player_name',
    'penalty_player_id',
    'penalty_player_name',
    'drive_quarter_end',
    'drive_time_of_possession',
    'drive_play_count',
    'drive_first_downs',
    'drive_inside20',
    'drive_ended_with_score',
    'drive_game_clock_end',
    'drive_end_yard_line',
    'drive_play_id_started',
    'drive_play_id_ended',
    'away_score',
    'home_score',
    'result',
    'total',
    'total_line',
    'spread_line',
    'passer',
    'passer_jersey_number',
    'rusher',
    'rusher_jersey_number',
    'receiver',
    'receiver_jersey_number',
    'passer_id',
    'rusher_id',
    'receiver_id',
    'name',
    'jersey_number',
    'id',
    'fantasy_player_id',
    'fantasy_player_name',
    'fantasy',
    'fantasy_id',
    'qb_epa',
    'xyac_epa',
    'pass_oe',

    'nfl_api_id',
    'play_clock',
    'play_deleted',
    'st_play_type',
    'end_clock_time',
    'end_yard_line',
    'drive_end_transition',
    'drive_real_start_time',
    'drive_yards_penalized',
    'drive_end_transition',
    'play_type_nfl',
    'stadium',
    'weather',
    'field_goal_result',
    'kick_distance',
    'extra_point_result',
    'two_point_conv_result',
    'timeout',
    'td_team',
    'td_player_id',
    'td_player_name',
    'lateral_receiving_yards',
    'lateral_rushing_yards',
    'forced_fumble_player_1_team',
    'forced_fumble_player_2_team',
    'solo_tackle_1_team',
    'solo_tackle_2_team',
    'assist_tackle_1_team',
    'assist_tackle_2_team',
    'tackle_with_assist',
    'tackle_with_assist_1_team',
    'tackle_with_assist_2_team',
    'fumbled_1_team',
    'fumbled_2_team',
    'fumble_recovery_1_team',
    'fumble_recovery_2_team',
    'fumble_recovery_1_yards',
    'fumble_recovery_2_yards',
    'return_team',
    'replay_or_challenge_result',
    'safety_player_name',
    'safety_player_id',
    'season',
    'series_success',
    'series_result',
    'order_sequence',
    'start_time',
    'drive_start_yard_line',
    'location',
    'timeout_team',
    'interception',
    'fourth_down_failed',
    'punt_blocked',
    'touchback',
    'punt_inside_twenty',
    'punt_in_endzone',
    'punt_out_of_bounds',
    'punt_downed',
    'punt_fair_catch',
    'kickoff_inside_twenty',
    'kickoff_in_endzone',
    'kickoff_out_of_bounds',
    'kickoff_downed',
    'kickoff_fair_catch',
    'fumble_forced',
    'fumble_not_forced',
    'fumble_out_of_bounds',
    'safety',
    'fumble_lost',
    'own_kickoff_recovery',
    'own_kickoff_recovery_td',
    'touchdown',
    'pass_touchdown',
    'rush_touchdown',
    'return_touchdown',
    'extra_point_attempt',
    'two_point_attempt',
    'field_goal_attempt',
    'kickoff_attempt',
    'punt_attempt',
    'fumble',
    'assist_tackle_3_team',
    'assist_tackle_4_team',
    'return_yards',
    'defensive_two_point_attempt',
    'defensive_two_point_conv',
    'defensive_extra_point_attempt',
    'defensive_extra_point_conv',
    'cp',
    'cpoe',
]

columns_to_fill = ['defteam_score', 'score_differential', 'home_wp_post', 'away_wp_post', 'down',
                            'kick_distance', 'passing_yards', 'receiver_player_id', 'receiver_player_name',
                            'receiving_yards', 'rusher_player_id', 'rusher_player_name', 'rushing_yards',
                            'punt_blocked', 'first_down_rush', 'first_down_pass', 'first_down_penalty',
                            'third_down_converted', 'third_down_failed', 'fourth_down_converted',
                            'fourth_down_failed', 'incomplete_pass', 'touchback', 'interception',
                            'punt_inside_twenty', 'punt_in_endzone', 'punt_out_of_bounds', 'punt_downed',
                            'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone',
                            'kickoff_out_of_bounds', 'kickoff_downed', 'kickoff_fair_catch', 'fumble_forced',
                            'fumble_not_forced', 'fumble_out_of_bounds', 'solo_tackle', 'safety', 'penalty',
                            'tackled_for_loss', 'fumble_lost', 'own_kickoff_recovery', 'own_kickoff_recovery_td',
                            'qb_hit', 'rush_attempt', 'pass_attempt', 'sack', 'touchdown', 'pass_touchdown',
                            'rush_touchdown', 'return_touchdown', 'extra_point_attempt', 'two_point_attempt',
                            'field_goal_attempt', 'kickoff_attempt', 'punt_attempt', 'fumble', 'complete_pass',
                            'assist_tackle', 'lateral_reception', 'lateral_rush', 'lateral_return', 'lateral_recovery',
                            'defensive_two_point_attempt', 'defensive_two_point_conv', 'defensive_extra_point_attempt',
                            'defensive_extra_point_conv', 'penalty_yards', 'cp', 'cpoe', 'wind', 'xyac_mean_yardage',
                            'xyac_median_yardage', 'xyac_success', 'xyac_fd', 'xpass', 'air_yards', 'yards_after_catch',
                            'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining',
                            ]

columns_to_fill_with_none = ['penalty_team', 'penalty_type', 'pass_length', 'pass_location', 'run_location', 'run_gap',]

temp_columns_to_drop = [
    'xyac_mean_yardage',
    'xyac_median_yardage',
    'xyac_success',
    'xyac_fd',
    'xpass',
]


In [ ]:
# normalize time, time_of_day and drive_game_clock_start
def normalize_time(df):
    for column in ['time', 'time_of_day', 'drive_game_clock_start']:
        df[column] = df[column].apply(lambda x: str(x).split(':')[0])
    return df

df = normalize_time(df)

In [ ]:
# function to normalize seconds remaining
def seconds_remaining(sr, total_seconds):
    if sr > total_seconds - 100:
        sr_range = f"{total_seconds - 99}-{total_seconds}"
    else:
        sr_range = f"{sr // 100 * 100 + 1}-{(sr // 100 + 1) * 100}"
    return sr_range

In [ ]:
# normalize seconds remaining
def normalize_seconds_remaining(df):
    for key, total_seconds in zip(['quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining'], [900, 1800, 3600]):
        df[key] = df[key].apply(lambda x: seconds_remaining(x, total_seconds))

    return df

In [ ]:
# normalize side_of_field
def normalize_side_of_field(df):
    df['side_of_field'] = df['side_of_field'].apply(
        lambda x: 'home' if x == df['home_team'].values[0] else 'away'
    )
    return df

In [ ]:
# normalize yardline_100
def normalize_yardline_100(df):
    df['yardline_100'] = df['yardline_100'].apply(
        # make yardlines intervals of 10
        lambda x: f"{x // 10 * 10 + 1}-{(x // 10 + 1) * 10}"
    )
    return df

In [ ]:
# normalize yrdln
def normalize_yrdln(df):
    # replace team abbreviations with home and away
    yard_num = 0
    df['yrdln'] = df['yrdln'].apply(
        # split yrdln into team and yardline delimited by space
        lambda x: x.split(' ')
    ).apply(
        # replace team abbreviations with home and away
        lambda x: (f'home {x[1]}') if x[0] == df['home_team'].values[0] else (f'away {x[1]}')
    )
    return df

In [ ]:
# normalize defteam
def normalize_defteam(df):
    # replace team abbreviations with home and away
    df['defteam'] = df['defteam'].apply(
        lambda x: 'home' if x == df['home_team'].values[0] else 'away'
    )
    return df

In [ ]:
# drop the columns that are not needed
df[columns_to_fill] = df[columns_to_fill].fillna(0)
df[columns_to_fill_with_none] = df[columns_to_fill_with_none].fillna('None')
df.drop(columns=temp_columns_to_drop, inplace=True)
df.drop(columns=columns_to_be_dropped, inplace=True)
df.dropna(inplace=True)
df = normalize_time(df)
df = normalize_seconds_remaining(df)
df = normalize_side_of_field(df)
df = normalize_yardline_100(df)
df = normalize_yrdln(df)
df = normalize_defteam(df)
df.head()

In [ ]:
# # write the df to a csv to check
# df.to_csv('./data.csv', index=False)

In [ ]:
from sklearn.model_selection import train_test_split
# shuffle the data
df = df.sample(frac=1, random_state=42)

# split the data into train and test
target = df['fixed_drive_result']
df.drop(columns=['fixed_drive_result'], inplace=True)

target = target.apply(
    lambda x: 0 if x == 'End of half' else (
        1 if x == 'Field goal' else (
            2 if x == 'Missed field goal' else (
                3 if x == 'Opp touchdown' else (
                    4 if x == 'Punt' else (
                        5 if x == 'Safety' else (
                            6 if x == 'Touchdown' else (
                                7 if x == 'Turnover' else (
                                    8 if x == 'Turnover on downs' else x
                                )
                            )
                        )
                    )
                )
            )
        )
    )
)

# drop dummies
# df = pd.get_dummies(df, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.2, random_state=42)
X_train = pd.get_dummies(X_train, drop_first=True)
# drop dummies from test set but keep the same columns
X_test = pd.get_dummies(X_test, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

In [ ]:
# write the df to a csv to check
df.to_csv('./data.csv', index=False)

In [ ]:
# Build the model with SVM and XGBoost
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pickle

# # SVM
# # define the model
# model = SVC()
# # fit the model
# model.fit(X_train, y_train)
# # predict the model
# y_pred = model.predict(X_test)
# # evaluate the model
# # print(accuracy_score(y_test, y_pred))
# # print(confusion_matrix(y_test, y_pred))
# # print(classification_report(y_test, y_pred))
# # write evaluation to a file
# with open('./evaluation.txt', 'w') as f:
#     f.write(f"Accuracy Score: {accuracy_score(y_test, y_pred)}\n")
#     f.write(f"Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}\n")
#     f.write(f"Classification Report: \n{classification_report(y_test, y_pred)}\n")

# # save the model to disk in .pkl format
# filename = 'svm_model.pkl'
# pickle.dump(model, open(filename, 'wb'))

# XGBoost
# define the model
model = xgb.XGBClassifier()
# set the parameters
parameters = {
    'max_depth': 3,
    'n_estimators': 100,
    'learning_rate': 0.2,
    'gamma': 0.1,
    'verbosity': 0,
    'objective': 'multi:softmax',
    'num_class': 9,
}
model.set_params(**parameters)
# fit the model
model.fit(X_train, y_train, verbose=True, eval_metric=['mlogloss', 'merror'], eval_set=[(X_train, y_train), (X_test, y_test)])
# predict the model
y_pred = model.predict(X_test)
# evaluate the model
cm = pd.DataFrame(confusion_matrix(y_test, y_pred), columns=['Predicted End of half', 'Predicted Field Goal', 'Predicted Missed field goal', 'Predicted Opp touchdown', 'Predicted Punt', 'Predicted Safety', 'Predicted Touchdown', 'Predicted Turnover', 'Predicted Turnover on downs'], index=['Actual End of half', 'Actual Field Goal', 'Actual Missed field goal', 'Actual Opp touchdown', 'Actual Punt', 'Actual Safety', 'Actual Touchdown', 'Actual Turnover', 'Actual Turnover on downs'])
cr = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose()
# write evaluation to a file
with open('./evaluation.txt', 'w') as f:
    f.write(f"Accuracy Score: {accuracy_score(y_test, y_pred)}\n")
    f.write(f"Confusion Matrix: \n{cm}\n")
    f.write(f"Classification Report: \n{cr}\n")

# save the model to disk
filename = 'xgb_model.pkl'
pickle.dump(model, open(filename, 'wb'))